In [ ]:
%reload_ext autoreload
%autoreload 2

In [92]:
import json

def pretty(d):
    print(json.dumps(d, indent=2, sort_keys=True, default=lambda o: str(o)))

In [47]:

import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '/home/gabriel/credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

# Retrieving schedule calendars

The filter by the prefix is just for illustration. Each created schedule would be stored in the database pointing it to the worker it related to.

In [75]:
all_calendars = service.calendarList().list().execute()
schedule_name_prefix = "[RDC]"

for calendar in all_calendars['items']:
    if calendar['summary'].startswith(schedule_name_prefix):
        print(f"{calendar['summary']}: {calendar['id']}")

worker_schedules = [
    {'id': calendar['id']}
    for calendar in all_calendars['items']
    if calendar['summary'].startswith(schedule_name_prefix)
]

[RDC] Agenda do Washington: h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com
[RDC] Agenda do Robson: 8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com


# Creating a schedule for a worker

In [70]:
calendars_service = service.calendars()
timezone = 'America/Sao_Paulo'
body = {
    "kind": "calendar#calendar",
    "summary": f"{schedule_name_prefix} Agenda do Robson",
    "timeZone": timezone
}
request = calendars_service.insert(body=body)
# request.execute()

{'kind': 'calendar#calendar',
 'etag': '"TtCNGp6X2MOylShgi0mPuHKaiRY"',
 'id': '8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com',
 'summary': '[RDC] Agenda do Robson',
 'timeZone': 'America/Sao_Paulo',
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['eventHangout']}}

# Getting free time slots from workers

In [86]:
import arrow, datetime

timezone = 'America/Sao_Paulo'
working_day_start = arrow.get(datetime.datetime(2020, 3, 19, 18), timezone)
working_day_end = arrow.get(datetime.datetime(2020, 3, 19, 21, 30), timezone)

freebusy_response = service.freebusy().query(body={
    "calendarExpansionMax": len(worker_schedules),
    "groupExpansionMax": 0,
    "timeMax": working_day_end.isoformat(),
    "items": worker_schedules,
    "timeMin": working_day_start.isoformat(),
    "timeZone": timezone
}).execute()

def busy_item_to_dates(busy_item):
    return {
        key: arrow.get(value)
        for key, value in busy_item.items()
    }

busy_times_by_calendar = {
    calendar_id: [busy_item_to_dates(busy_item) for busy_item in calendar['busy']]
    for calendar_id, calendar in freebusy_response['calendars'].items()
}

In [93]:
pretty(freebusy_response)

{
  "calendars": {
    "8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com": {
      "busy": []
    },
    "h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com": {
      "busy": [
        {
          "end": "2020-03-19T20:30:00-03:00",
          "start": "2020-03-19T19:30:00-03:00"
        }
      ]
    }
  },
  "kind": "calendar#freeBusy",
  "timeMax": "2020-03-20T00:30:00.000Z",
  "timeMin": "2020-03-19T21:00:00.000Z"
}


In [94]:
pretty(busy_times_by_calendar)

{
  "8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com": [],
  "h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com": [
    {
      "end": "2020-03-19T20:30:00-03:00",
      "start": "2020-03-19T19:30:00-03:00"
    }
  ]
}


In [95]:
def is_slot_available(busy_times_by_calendar, start_time, duration_in_minutes):
    pass